# François' Dev Notebook 1


### Un pipe mimimaliste
L'objectif de ce notebook est de mettre en place un pipe qui prenne en input un jeu de données patient avec chacun une pile de dicom et un filtre assoscié afin d'en extraire les features.

### Low level implementation scheme

```
Pour chaque dossier dans le dossier data:
    Instancier un objet patient avec list_lesions vide
    ajouter le patient à la liste de patients
    Ouvrir le dossier dcm du dossier patient:
        convertir la pile DCM sous la forme d'un voxel ndarray puis d'un sITK
        Ajouter le sITK créer comme attribut image du patient
    Pour chaque dosier dans le dossier patient:
        Si le dossier a pour nom 'lx' avec x un entier:
            Instancier un nouvel objet lésion:
               Mettre l'attribut nom de la lésion comme le nom du dossier lx
               Création du masque retenue par vote majoriateire
               Mettre l'attribut mask de la lesions sous la forme d'un sITK
            Superposer le masque et la lésion
            Extraire les caractéristiques radiomics de la superposition
            Set l'attribut features de la lesions
            
    
    
```

In [3]:
import numpy as np
import dicom
import dicom_numpy
import os
from radiomics import featureextractor
from skimage import io
import SimpleITK as sitk
import six
import re
from skimage.external import tifffile

In [4]:
PATH_TO_DATA  = "/Users/pops/Documents/ecn/projet/MYELOME/" 

## Quelques fonctions intermédiaires 
--> a déplacer dans un fichier utils que l'on appelera depuis le fichier principale

### Fonction dcmToSimpleITK
Pour traiter une pile de dicom et en donner une image simpleITK qui peut être utilisé par pyradiomics

### Fonction majorityVote
A partir de 3 masques au format .tif, retourne le filtre sélectionné par la méthode du vote majoritaire, ce masque est retournée sous forme d'une image simpleITK qui peut être utilisé par pyradiomics

### Fonction getTifMasks
A partir d'un dossier lésion, la fonction getTifMasks renvoie les 3 masques 40, 2.5 et kmean au format .tif 

### Fonction makeTifFromPile 
A partir d'une pile de masque, la fonction makeTifFromPile créer un masque au format.tif et le place dans le dossier racine de la lésion. Cette fonction prend en entrée le chemin vers la pile de fichiers à convertir en .tif. makeTifFromPile retourne le chemin vers le nouveau masque au format tif créé

In [5]:
def computeSUVFactor(dcmSlice):
    '''Calculate the SUV factor used to normalized a patient dcm pile in SUV'''
    
    # Check the delay between injection and acquisition
    # Get infos from the patient dcm tags
    acquisitionHour = int(dcmSlice[0x00080031].value[0:2])
    acquisitionMinute = int(dcmSlice[0x00080031].value[2:4])
    injectionHour = int(dcmSlice[0x00540016].value[0][0x00181072].value[0:2])
    injectionMinute = int(dcmSlice[0x00540016].value[0][0x00181072].value[2:4])
    
    deltaHour = acquisitionHour - injectionHour
    deltaMinute = acquisitionMinute - injectionMinute
    
    if (deltaMinute < 0):
        deltaMinute = 60 + deltaMinute
        deltaHour = deltaHour - 1
    
    delay = deltaHour*60 + deltaMinute
    print("Delay: %i min" % delay)
    
    if (delay < 55) or (delay > 75):
        print("*****************************************************")
        print("WARNING!!!! Delay not in the range of EANM guidelines")
        print("*****************************************************")
    
    
    #----------------------
    # Compute the suvFactor
    #----------------------
        
    manufacturer = dcmSlice[0x00080070].value.lower()
    manufacturerIsPhilips = "philips" in manufacturer
    
    units = dcmSlice[0x00541001].value.lower()
    unitIsNotBqml = "bqml" not in units
    
    if (manufacturerIsPhilips) and (unitIsNotBqml):
        SUVfactor = float(dcmSlice[0x70531000])
        
        
    
    print(manufacturer)

    
    suvFactor = 1
    
    return suvFactor

Pipe de conversion en Suv

Lors de l'ouverture de la pile de dicom du patient :
    - Controle l'unité de la pile de dcm
    Si c'est bonne unité on continue le pipe, sinon on rescale 
    
On parcourt chaque 
    

Transformation d'une pile en dcm 
    

In [6]:
pathToSlice = os.path.join(PATH_TO_DATA,"050-004/dcm/PI.1.3.46.670589.28.2.15.2199422639.3.82.0.13067747800.dcm")
dcmSlice = dicom.read_file(pathToSlice)
print(dcmSlice.PatientName,
     dcmSlice.RescaleSlope)
print(dcmSlice[0x00100010])
print(dcmSlice[0x18,0x50])
print(dcmSlice)
print("######## HERE ########")
radioPharmaceutical = dcmSlice[0x00540016]

suvFactor = computeSUVFactor(dcmSlice)
print("suv factor : %i" % suvFactor)

PatientAnonyme_20110728151314 1
(0010, 0010) Patient's Name                      PN: 'PatientAnonyme_20110728151314'
(0018, 0050) Slice Thickness                     DS: '4'
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0016) SOP Class UID                       UI: Positron Emission Tomography Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.46.670589.28.2.15.2199422639.3.82.0.1306774780
(0008, 0020) Study Date                          DA: '20110530'
(0008, 0021) Series Date                         DA: '20110530'
(0008, 0022) Acquisition Date                    DA: '20110530'
(0008, 0023) Content Date                        DA: '20110530'
(0008, 0030) Study Time                          TM: '171318.000000'
(0008, 0031) Series Time                         TM: '171318.000000'
(0008, 0032) Acquisition Time                    TM: '171318.000000'
(0008, 0033) Content

TypeError: float() argument must be a string or a number, not 'DataElement'

In [7]:
def dcmToSimpleITK(dcmDirectory):
    '''Return sITK type image from a pile of dcm files'''
    list_dcmFiles = []
    for directory, subDirectory, list_dcmFileNames in os.walk(dcmDirectory):
        for dcmFile in list_dcmFileNames:
            list_dcmFiles.append(os.path.join(directory, dcmFile))
    dcmImage = [dicom.read_file(file) for file in list_dcmFiles]
    voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(dcmImage)
    sITK_image = sitk.GetImageFromArray(voxel_ndarray)
    return(sITK_image)
    

#2 Fonction intermediaire getTifMasks
```
Ouverture du dossier lesion:

Si il existe un fichier 25.tif:
    assigner le chemin vers ce fichier à la variable pathToMask25Tif
Sinon ouvrir le dossier 25:
    construire .tif à partir de la pile
    assigner le chemin vers ce fichier à la variable pathToMask25Tif
    
Si il existe un fichier 40.tif:
    assigner le chemin vers ce fichier à la variable pathToMask40Tif
Sinon ouvrir les dossiers 40 
    construire .tif à partir de la pile
    assigner le chemin vers ce fichier à la variable pathToMask40Tif
    
retourner les 3 masques en .tif
```

In [8]:
def getTifMasks(masksPath):
    '''Return path toward the KMean, 40 and 2.5 mask respectively in this order'''
    list_files = [file for file in os.listdir(masksPath) if os.path.isfile(os.path.join(masksPath, file))]
    mask40Name = '40.tif' 
    mask25Name = '25.tif' 
    pathToKmeanMask = masksPath + '/kmean.tif'
    if mask40Name in list_files:
        pathTo40Mask = os.path.join(masksPath, mask40Name)
        print(pathTo40Mask)
    else:
        pathTo40Mask = makeTifFromPile(os.path.join(masksPath, "40"))
    if mask25Name in list_files:
        pathTo25Mask = os.path.join(masksPath, mask25Name)
    else:
        pathTo25Mask = makeTifFromPile(os.path.join(masksPath, '2.5'))
    return(pathToKmeanMask, pathTo40Mask, pathTo25Mask)

In [9]:
def getWords(text):
    return re.compile('\w+').findall(text)

In [10]:
def makeTifFromPile(pathToPile):
    '''Takes an absoulte path containing a pile of masks, compute the resulting .tif mask and output the path to the created .tif mask '''
    list_pileFiles = []
    for dirpath,dirnames,fileNames in os.walk(pathToPile):
        for file in fileNames:
            list_pileFiles.append(os.path.join(dirpath,file))
    list_pileFilesAsArray = []
    
    first_file = list_pileFiles[0]
    #get the shape of the image
    with open(first_file, mode='r', encoding='utf-8') as file:
        file.readline()#first line is junk data
        shapeLocalMask = getWords(file.readline()) #secondline is shape of the dcm pile
        xShape = int(shapeLocalMask[0])
        yShape = int(shapeLocalMask[1])
        mask_size = (xShape, yShape)
    
    num_file = len(list_pileFiles)
    mask_array = np.zeros((num_file, xShape, yShape))
    fileIndex = num_file-1
    for pileFile in list_pileFiles:
        with open(pileFile, mode='r', encoding='utf-8') as file:
            file.readline() #junk line
            file.readline() #secondline is shape of the dcm pile
            file.readline() # 3 lines of junk data
            for rowIndex in range(xShape):
                for colIndex in range(yShape):
                    val = file.read(1)
                    while (val!='0' and val!='1'):
                        val = file.read(1)
                    mask_array[fileIndex,rowIndex, colIndex] = int(val)
            fileIndex = fileIndex - 1

    pathToLesion = os.path.abspath(os.path.join(pathToPile, os.pardir))

    if('2.5' in pathToPile):
        pathToTifMask = os.path.join(pathToLesion, '25.tif')
    if('40' in pathToPile):
        pathToTifMask = os.path.join(pathToLesion, '40.tif')
        
    tifffile.imsave(pathToTifMask, mask_array)
    return pathToTifMask

In [11]:
masksPath = PATH_TO_DATA + "001-060/l1"
getTifMasks(masksPath)

/Users/pops/Documents/ecn/projet/MYELOME/001-060/l1/40.tif


('/Users/pops/Documents/ecn/projet/MYELOME/001-060/l1/kmean.tif',
 '/Users/pops/Documents/ecn/projet/MYELOME/001-060/l1/40.tif',
 '/Users/pops/Documents/ecn/projet/MYELOME/001-060/l1/25.tif')

In [12]:
maskKmean = tifffile.imread(PATH_TO_DATA + '001-060/l1/kmean.tif')
mask40 = tifffile.imread(PATH_TO_DATA + '001-060/l1/40.tif')


In [13]:
print('Kmean : ',maskKmean.shape,' \n40 : ', mask40.shape)

Kmean :  (311, 128, 128)  
40 :  (374, 168, 168)


### majorityVote implementation

Le calcul du mask résultant par vote_maj se fait sur des fichiers en .tif

In [14]:
def majorityVote(masksPath):
    print(masksPath)
    '''Compute the average mask based on the majority vote method from different masks from a lesion'''
    #TO DO : Add a a safety check to make sure weuse the file Result.tiff, if it exists in the masksPath directory. 
    # this mask corresponds to the resulting voteMaj mask. No need to recompute it.
    
    # Import des 3 masques tif
    (pathToKmeanMask, pathTo40Mask, pathTo25Mask) = getTifMasks(masksPath)

    
    mkmean = io.imread(pathToKmeanMask).T
    m25 = io.imread(pathTo40Mask).T
    m40 = io.imread(pathTo25Mask).T
    
    
    # Paramètres
    ACCEPT = 0.33  # seuil d'acceptation pour que le voxel appartienne à la lésion ou au fond, valeur donnée par Thomas 0.45
    nbmethods = 3  # Nombre de methodes (40%, 2.5 et kmean)
    
    # Dimensions
    imageDims = mkmean.shape
    
    # Image binaire segmentée selon vote majoritaire
    mMajority = np.zeros(imageDims)

    somme = m40 + m25 + mkmean
    somme /= nbmethods

    mMajority[somme >= ACCEPT] = 1 #Vectorized method
    
    # print("somme shape", somme.shape) #element wise method
    # for xindex, x in enumerate(somme):
    #     for yindex, y in enumerate(x):
    #         for zindex, z in enumerate(y):
    #             #if z != 0:
    #              #   print(z)
    #               #  mMajority[xindex, yindex, zindex] = 1
    #             if z >= ACCEPT:
    #                 mMajority[xindex, yindex, zindex] = 1

    
    sITK_mask = sitk.GetImageFromArray(mMajority)
    
    majorityTiffPath = os.path.join(masksPath, "majority.tif")
    tifffile.imsave(majorityTiffPath, mMajority)
    
    return sITK_mask

In [15]:
class Patient:

    def __init__(self, ref, image=None, list_lesions=[]):
        '''Provide ref number of the patient as a string "xxx-xxx", image must be simpleITK'''
        self.ref = ref
        self.list_lesions = list_lesions
        dcmDirectory = os.path.join(PATH_TO_DATA, ref, "dcm")
        self.image = dcmToSimpleITK(dcmDirectory)
        

In [16]:
class Lesion:
    
    def __init__(self, ref, masksPath):
        '''Provide ref number as string "lx", where x is the number of the lesion, masksPath is the path to the folder conatining the masks'''
        self.ref = ref
        self.mask = majorityVote(masksPath)
        self.dico_features = {} #la liste des features à récupérer sera à définir avec Thomas

In [17]:
PARAM_PATH = "/Users/pops/Documents/ecn/projet/petml/code/extractionParams.yaml"

In [18]:
#test du focntionnement de l'extarctionà partir d'un fichier de paramètres en .yaml
# objectif d'intancier un extracteur auquel on fourni la liste des features à extraire
# en dehors du code pour plus de flexibilité.
ref = "050-004"
patient1 = Patient(ref)
patient1.ref
patient1.image
#lesion1 = Lesion("l2", "/Users/pops/Documents/ecn/projet/MYELOME/001-060/l2")
print('Image patient  %s' % type(patient1.image))
#print('Image lésion %s' % type(lesion1.mask))
mask = sitk.GetImageFromArray(io.imread("/Users/pops/Documents/ecn/projet/MYELOME/" + ref + "/l2/kmean.tif").T)
#lesion1.mask = sitk.GetImageFromArray(io.imread("/Users/pops/Documents/ecn/projet/MYELOME/001-060/l2/kmean.tif").T)
print("Superposition des masques")



#extractor = featureextractor.RadiomicsFeaturesExtractor(PARAM_PATH)
###


# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeaturesExtractor(PARAM_PATH)
extractor.loadParams(paramsFile=PARAM_PATH)
classNames = extractor.getFeatureClassNames()
#for value in iter(classNames):
    #print(value)
    #print(extractor.getFeatureNames(value))
    #print(extractor.computeFeatures(patient1.image, lesion1.mask))
    #print(extractor.execute(patient1.image, mask))
features = extractor.execute(patient1.image, mask)
for key in features:
    print("%s: %s \n" % (key, features[key]))

Image patient  <class 'SimpleITK.SimpleITK.Image'>
Superposition des masques
general_info_BoundingBox: (298, 91, 71, 4, 9, 7) 

general_info_EnabledImageTypes: {'Original': {}} 

general_info_GeneralSettings: {'minimumROIDimensions': 1, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'enableCExtensions': True, 'additionalInfo': True, 'binWidth': 3, 'weightingNorm': None} 

general_info_ImageHash: 15d6a046a3b8a135f800417307c00f8dcc344c1c 

general_info_ImageSpacing: (1.0, 1.0, 1.0) 

general_info_MaskHash: f7a3ab77eb5f0ebec567533b7bd30ede4e048203 

general_info_NumpyVersion: 1.13.3 

general_info_PyWaveletVersion: 0.5.2 

general_info_SimpleITKVersion: 1.0.1 

general_info_Version: 1.3.0.post53+g8c7dd45 

general_info_VolumeNum: 1 

general_info_VoxelNum: 1

### Fonction extraction des features pour une lésion

In [19]:
def setFeatures(dico_features, image, mask, paramPath):
    
    # Instantiate the extractor
    extractor = radiomics.featureextractor.RadiomicsFeaturesExtractor(paramPath)
    R = radiomics.glcm.RadiomicsGLCM(image, mask)
    dico_features["Autocorrelation"] = R.getAutocorrelationFeatureValue()
    dico_features["ClusterProminence"] = R.getClusterProminenceFeatureValue()
    dico_features["ClusterShade"] = R.getClusterShadeFeatureValue()
    dico_features["Contrast"] = R.getContrastFeatureValue()
    dico_features["Correlation"] = R.getCorrelationFeatureValue()
    dico_features["DifferenceEntropy"] = R.getDifferenceEntropyFeatureValue()
    dico_features["DifferenceVariance"] = R.getDifferenceVarianceFeatureValue()
    
    return dico_features

## Pipeline pour l'extraction automatisée de features
A partir d'un dossier de patients suivant une architecture standardisée

In [20]:
list_patients = []
for refPatient in os.listdir(PATH_TO_DATA):
    print("Processing patient %s ..." % refPatient)
    patient = Patient(refPatient)
    list_patients.append(patient)
    patient.list_lesions = []
    for directoryName in os.listdir(os.path.join(PATH_TO_DATA, patient.ref)):
        if directoryName != 'dcm' and directoryName != []:
            print("Processing lesion %s ..." % directoryName)
            masksPath = os.path.join(PATH_TO_DATA, refPatient, directoryName)
            lesion = Lesion(directoryName, masksPath)
            patient.list_lesions.append(lesion)

            lesion.dico_features = setFeatures(lesion.dico_features, patient.image, lesion.mask, PARAM_PATH)
            

Processing patient completeData10_January2018_4ECN.csv ...


DicomImportException: Must provide at least one DICOM dataset

_NB_  : Je suppose un bug sur la lésion l1 du patient 001-026, certaines valeurs du masque doivent être incohérentes car tous les autres masques fonctionnnent. Pour mes tests, j'ai déplacé cette data dans un donnée corrupted_data

In [ ]:
for patient in list_patients:
    for lesion in patient.list_lesions:
        print(patient.ref)
        print(lesion.ref)
        print(lesion.dico_features)